## Introduction

1. Based on the results from part 1, go to the appropriate website
area for treatment and scrape (using Scrapy)the text of treatment

2. Everything should be done automatically (especially for scraping,
system should be able to decide what area (text) of website to
scrape the text (based on the results)

  **=> Using a data from cancaer.ca as source of information than let the system scrapes the infromation from cancer.org and choose the desired position of each stage of cance by comparing the similarity using Google Universal sentences encoder.**

3. Process the text and using LSA, LDA and NNMF to extract the
important information

  **=> After geting the exactly information of cancer stage, this project uses LSA, LDA and NNMF to extract the important information by show the 10 words represented for this stage**

4. Using PageRank and Sequence2Sequence to summarize the
text information

  **=> Using TextRank to summarize the information with ratio 50%**

5. Do the same things for Medicine recommendation
  **=> didn't complete yet**

## Scraping Web

In [0]:
from bs4 import BeautifulSoup
# import dryscrape
# from selenium import webdriver # the website loads the data using JavaScript.Try use Selenium and beautiful soup.
import requests
import pandas as pd
import numpy as np

The imported “request” library has a get() function which will request the indeed.com server for the content of the URL and store the server’s response in the “base_url” variable. If we print the “base_url” variable we’ll actually see the entire HTML for the page.

In [0]:
test_base_url = requests.get("https://www.cancer.org/cancer/lung-cancer/treating-non-small-cell/by-stage.html", timeout=5)
data_base_url = "https://www.cancer.ca/en/cancer-information/cancer-type/lung/treatment/"

In [3]:
test_base_url.text[-2000:-2301:-1]

'bal-aira "ladoMcireneg"=di "ladom-cireneg edaf ladom"=ssalc vid<    \n\r>vid/<\n\r>vid/<    \n\r>vid/<        \n\r>vid/<            \n\r>vid/<                \n\r>a/<>naps/<>naps/<txeN>"ylno-rs"=ssalc "eurt"=neddih-aira naps<>"2thgir-worra-oci oci"=ssalc naps<>"txen"=edils-atad "lesuoraCladom#"=ferh "thgir lortn'

- Defining a function named “parse” which will take one argument which will be the actual URL of the page we are attempting to parse/scrape.
- Use the BeautifulSoup class crator to parse the content (HTML code) of the website we are trying to parse.
-We’ll use the “lxml” parser just in case the HTML is not perfectly formed.

In [0]:
page_content = BeautifulSoup(test_base_url.content, 'html.parser')

In [5]:
page_content.findAll('h1')

[<h1>Treatment Choices for Non-Small Cell Lung Cancer, by Stage</h1>]

In [6]:
page_content.findAll('h2')

[<h2>Treating occult cancer</h2>,
 <h2>Treating stage 0 NSCLC</h2>,
 <h2>Treating stage I NSCLC</h2>,
 <h2>Treating stage II NSCLC</h2>,
 <h2>Treating stage IIIA NSCLC</h2>,
 <h2>Treating stage IIIB NSCLC</h2>,
 <h2>Treating stage IV NSCLC</h2>,
 <h2>NSCLC that progresses or recurs after treatment</h2>,
 <h2><div class="inner"><span class="left-bar"></span>More In Lung Cancer<span class="right-bar"></span></div></h2>]

In [0]:
# class="col-md-12" # whole page
# class="text-ckeditor section" # text under titile
def parse_test(full_url):
  page_content = BeautifulSoup(full_url.content, 'html.parser')
  containers = page_content.find('body', { 'class': None})
  df = pd.DataFrame(columns = ['pargraphs', 'stages'])
  print(len(containers.findAll('p')))
  for section in containers.findAll('h2'):
    nextNode = section
    pargraph = nextNode
    print(nextNode)
    while True:
        nextNode = nextNode.nextSibling
        if nextNode == "\n":
              continue
        try:            
            tag_name = nextNode.name
        except AttributeError:
            tag_name = ""
        if tag_name == "p":
            print(nextNode.text)
            #print(nextNode.string)# error not show all <p>
            df = df.append({'pargraphs': nextNode.text, 'stages' : pargraph.text}, ignore_index = True)
        else:
            print ("*****")
            break
  return df

In [8]:
df_ScrapeTreatment = parse_test(test_base_url)

39
<h2>Treating occult cancer</h2>
For these cancers, malignant cells are seen on sputum cytology but no obvious tumor can be found with bronchoscopy or imaging tests. They are usually early-stage cancers. Bronchoscopy and possibly other tests are usually repeated every few months to look for a tumor. If a tumor is found, treatment will depend on the stage.
*****
<h2>Treating stage 0 NSCLC</h2>
Because stage 0 NSCLC is limited to the lining layer of airways and has not invaded deeper into the lung tissue or other areas, it is usually curable by surgery alone. No chemotherapy or radiation therapy is needed.
If you are healthy enough for surgery, you can usually be treated by segmentectomy or wedge resection (removal of part of the lobe of the lung). Cancers in some locations (such as where the windpipe divides into the left and right main bronchi) may be treated with a sleeve resection, but in some cases they may be hard to remove completely without removing a lobe (lobectomy) or even a

In [11]:
df_ScrapeTreatment.tail()

,pargraphs,stages
24,If cancer continues to grow during treatment (...,NSCLC that progresses or recurs after treatment
25,If cancer continues to grow during initial tre...,NSCLC that progresses or recurs after treatment
26,Smaller cancers that recur locally in the lung...,NSCLC that progresses or recurs after treatment
27,"For more on dealing with a recurrence, see Und...",NSCLC that progresses or recurs after treatment
28,"In some people, the cancer may never go away c...",NSCLC that progresses or recurs after treatment


In [12]:
df_ScrapeTreatment.shape

(29, 2)

In [0]:
df_ScrapeTreatment["stages"].value_counts()

NSCLC that progresses or recurs after treatment    5
Treating stage II NSCLC                            5
Treating stage I NSCLC                             5
Treating stage IIIA NSCLC                          4
Treating stage 0 NSCLC                             3
Treating stage IV NSCLC                            3
Treating stage IIIB NSCLC                          3
Treating occult cancer                             1
Name: stages, dtype: int64

In [0]:
df_ScrapeTreatment[df_ScrapeTreatment["stages"] == 'Treating stage IV NSCLC']

,pargraphs,stages
21,Stage IV NSCLC is widespread when it is diagno...,Treating stage IV NSCLC
22,"If you are in otherwise good health, treatment...",Treating stage IV NSCLC
23,"Other treatments, such as photodynamic therapy...",Treating stage IV NSCLC


### Get Data to scrape a stage of cancer on the test web

In [0]:

def parse_data(full_url):
  stagesDict = {'stage-1': 'Treating stage I NSCLC', 'stage-2': 'Treating stage II NSCLC', 'stage-3': 'Treating stage III NSCLC'}
  df = pd.DataFrame(columns = ['pargraphs', 'stages'])
  for s in list(stagesDict.keys()): 
    base_url = full_url + s + "/?region=on"
    base_url = requests.get(base_url, timeout = 5)
    page_content = BeautifulSoup(base_url.content, 'html.parser')
    title = page_content.findAll('h1')[1].text
    containers = page_content.find('div', { 'class': "Section1"})
    print(s)
    flag = True
    # pargraphs = title
    for section in containers.findAll('h2'):
      nextNode = section
      print(nextNode)
      pargraphs =   " "
      if section.text == 'Clinical trials':
        continue
      if flag:
        flag = False
        pargraphs =   title + " "
      # if section.text not in ['Surgery', 'Radiation therapy', 'Chemotherapy', 'Clinical trials']:
      #   pargraphs = section.text + " "
      while True:
          nextNode = nextNode.nextSibling
          if nextNode == "\n":
                continue
          try:            
              tag_name = nextNode.name
          except AttributeError:
              tag_name = ""
          if tag_name == "p":
              print(nextNode.text)
              #print(nextNode.string)# error not show all <p>
              pargraphs +=  nextNode.text + " "
              
          else:
              df = df.append({'pargraphs': pargraphs, 'stages' : stagesDict[s]}, ignore_index = True)
              print ("*****")
              break
  return df

In [16]:
df_TreatmentData = parse_data(data_base_url)

stage-1
<h2>Surgery</h2>
Surgery is a standard treatment for stage 1 non–small cell lung cancer for people who are well enough to have surgery. 
Lobectomy to remove the lobe of the lung is the main type of surgery for stage 1 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed.
Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 1 non–small cell lung cancer in people who do not have very good lung function.
Sleeve resection is used to remove a tumour in the large airway of the lung (bronchus).
During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. The surgery may be stopped if there is cancer in more lymph nodes than was shown with diagnostic tests. This is done because the cancer has spread too far for surgery to be helpful as a treatment. 
Surgery may be done again if the lab reports that th

In [17]:
df_TreatmentData.shape

(12, 2)

**Only get 3 stages which are requirement for this project**

**Assume we get the diameter of cancer node and get the stage**

## Use Google Universal sentence encoder to find the smilar sentences or graphs

In [21]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

TensorFlow 1.x selected.


In [22]:
!mkdir test_SimlarTxt
# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" |   tar -zxvC test_SimlarTxt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 93  745M   93  698M    0     0  53.8M      0  0:00:13  0:00:12  0:00:01 56.3M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  54.3M      0  0:00:13  0:00:13 --:--:-- 56.7M


In [23]:
#Function so that one session can be called multiple times. 
#Useful while multiple calls need to be done for embedding. 
import tensorflow as tf
import tensorflow_hub as hub
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

embed_fn = embed_useT("test_SimlarTxt")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
Diameter = '3'
stage = 'Treating stage I NSCLC'
stage1 = df_TreatmentData[df_TreatmentData['stages'] == stage]['pargraphs']

Use information of stage 1 from source to find the place of stage 1 in the scraped web test

In [26]:
dataStage = []
for sx in stage1:
  dataStage.append(sx)
dataStage

['Treatments for stage 1 non–small cell lung cancer Surgery is a standard treatment for stage 1 non–small cell lung cancer for people who are well enough to have surgery.  Lobectomy to remove the lobe of the lung is the main type of surgery for stage 1 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed. Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 1 non–small cell lung cancer in people who do not have very good lung function. Sleeve resection is used to remove a tumour in the large airway of the lung (bronchus). During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. The surgery may be stopped if there is cancer in more lymph nodes than was shown with diagnostic tests. This is done because the cancer has spread too far for surgery to be helpful as a treatment.  Surgery may be done again

In [27]:
TestStage = []
for sx_test in df_ScrapeTreatment['pargraphs']:
  TestStage.append(sx_test)
print(len(TestStage))
TestStage[:5]

29


['For these cancers, malignant cells are seen on sputum cytology but no obvious tumor can be found with bronchoscopy or imaging tests. They are usually early-stage cancers. Bronchoscopy and possibly other tests are usually repeated every few months to look for a tumor. If a tumor is found, treatment will depend on the stage.',
 'Because stage 0 NSCLC is limited to the lining layer of airways and has not invaded deeper into the lung tissue or other areas, it is usually curable by surgery alone. No chemotherapy or radiation therapy is needed.',
 'If you are healthy enough for surgery, you can usually be treated by segmentectomy or wedge resection (removal of part of the lobe of the lung). Cancers in some locations (such as where the windpipe divides into the left and right main bronchi) may be treated with a sleeve resection, but in some cases they may be hard to remove completely without removing a lobe (lobectomy) or even an entire lung (pneumonectomy).',
 'For some stage 0 cancers, tr

In [28]:
messages = dataStage + TestStage
print(len(messages))
encoding_matrix = embed_fn(messages)
results = np.inner(encoding_matrix, encoding_matrix)
results[0]

32


array([0.9999999 , 0.8891977 , 0.8791368 , 0.81553197, 0.82436424,
       0.8344706 , 0.84074247, 0.86763126, 0.72092164, 0.81143546,
       0.84825206, 0.83177114, 0.8695761 , 0.7816832 , 0.86710036,
       0.80257547, 0.6717336 , 0.84305954, 0.86976993, 0.83490586,
       0.7133719 , 0.897699  , 0.8227962 , 0.7462059 , 0.7869357 ,
       0.65377605, 0.59027755, 0.8123313 , 0.8409171 , 0.85531604,
       0.3281222 , 0.76744276], dtype=float32)

In [0]:
subRes = results[0].copy()

index = [(i,x) for i, x in enumerate(subRes[3:]) if x > 0.89 ]

In [32]:
print(index)
messages[int(index[0][0]) +  3]

[(18, 0.897699)]


'Stage IIIB NSCLC has spread to lymph nodes that are near the other lung or in the neck, and may also have grown into important structures in the chest. These cancers can’t be removed completely by surgery. As with other stages of lung cancer, treatment depends on the patient’s overall health. If you are in fairly good health you may be helped by chemotherapy (chemo) combined with radiation therapy. Some people can even be cured with this treatment. If the cancer stays under control after 2 or more treatments of chemoradiation, the immunotherapy drug durvalumab can be given for up to a year to help keep the cancer stable.'

ERROR! Session/line number was not unique in database. History logging moved to new session 59


**=> data stage 1, but the test is stage 3b => not good measure**

In [0]:
##################################################################################|

**New solution: Concatenate all paragraphs of one stage into 1 paragraph and combine stage 3a and 3b => stage 3**

In [33]:
stage = 'Treating stage I NSCLC'
stage1 = df_TreatmentData[df_TreatmentData['stages'] == stage]['pargraphs']
stage1

0    Treatments for stage 1 non–small cell lung can...
1     Radiation therapy is offered for stage 1 non–...
2     Chemotherapy may be offered after surgery to ...
Name: pargraphs, dtype: object

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [34]:
stage = 'Treating stage II NSCLC'
stage2 = df_TreatmentData[df_TreatmentData['stages'] == stage]['pargraphs']
stage2

3    Treatments for stage 2 non–small cell lung can...
4     External beam radiation therapy is offered fo...
5     Chemotherapy may be offered after surgery for...
6     Chemoradiation may be offered as a treatment ...
Name: pargraphs, dtype: object

In [35]:
stage = 'Treating stage III NSCLC'
stage3 = df_TreatmentData[df_TreatmentData['stages'] == stage]['pargraphs']
stage3

7     Treatments for stage 3 non–small cell lung can...
8      The type of surgery done depends on where the...
9      The most common chemotherapy drug combination...
10     Imfinzi (durvalumab) may be offered as a trea...
11     External beam radiation therapy by itself may...
Name: pargraphs, dtype: object

In [38]:
dataStage1_v2 = ""
for x in list(stage1):
  dataStage1_v2 += x
dataStage1_v2

'Treatments for stage 1 non–small cell lung cancer Surgery is a standard treatment for stage 1 non–small cell lung cancer for people who are well enough to have surgery.  Lobectomy to remove the lobe of the lung is the main type of surgery for stage 1 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed. Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 1 non–small cell lung cancer in people who do not have very good lung function. Sleeve resection is used to remove a tumour in the large airway of the lung (bronchus). During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. The surgery may be stopped if there is cancer in more lymph nodes than was shown with diagnostic tests. This is done because the cancer has spread too far for surgery to be helpful as a treatment.  Surgery may be done again 

In [39]:
dataStage2_v2 = ""
for x in list(stage2):
  dataStage2_v2 += x
dataStage2_v2

'Treatments for stage 2 non–small cell lung cancer Surgery is a standard treatment for stage 2 non–small cell lung cancer for people who are well enough to have surgery. Lobectomy to remove the lobe of the lung is the main type of surgery for stage 2 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed.  Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 2 non–small cell lung cancer in people who do not have very good lung function. Sleeve resection is used to remove a tumour in the large airway (bronchus) of the lung. Extended pulmonary resection and a chest wall resection may be done for stage 2 non–small cell lung cancer that has spread to the chest wall or other tissues around the lung. During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. If there is cancer in more lymph nodes than shown

In [40]:
dataStage3_v2 = ""
for x in list(stage3):
  dataStage3_v2 += x
dataStage3_v2

'Treatments for stage 3 non–small cell lung cancer The chemotherapy drug combinations most often used with chemoradiation are cisplatin with etoposide (Vepesid, VP-16) or cisplatin with vinorelbine (Navelbine). Chemotherapy and external beam radiation therapy may be given together before surgery for stage 3A and 3B non–small cell lung cancer, in people who are well enough to have this treatment.  Chemoradiation may also be given to people who will not be having surgery and are well enough to have this treatment.  The type of surgery done depends on where the tumour is found in the lung. Surgery may include a lobectomy, a bilobectomy, a sleeve resection or a pneumonectomy. If the cancer has spread, an extended pulmonary resection or a chest wall resection may be done. Surgery for cancer that has grown into the bones of the spine is done using 2 surgeries, one to support the spine with rods followed by another surgery to remove the tumour.  For stage 3A non–small cell lung cancer, surger

In [0]:
TestStage_v2 = []
for st in df_ScrapeTreatment['stages'].unique():
  tempStr = ""
  for p in list(df_ScrapeTreatment[df_ScrapeTreatment['stages'] == st]['pargraphs']):
    tempStr += p
  TestStage_v2.append(tempStr)

In [42]:
TestStage_v2

['For these cancers, malignant cells are seen on sputum cytology but no obvious tumor can be found with bronchoscopy or imaging tests. They are usually early-stage cancers. Bronchoscopy and possibly other tests are usually repeated every few months to look for a tumor. If a tumor is found, treatment will depend on the stage.',
 'Because stage 0 NSCLC is limited to the lining layer of airways and has not invaded deeper into the lung tissue or other areas, it is usually curable by surgery alone. No chemotherapy or radiation therapy is needed.If you are healthy enough for surgery, you can usually be treated by segmentectomy or wedge resection (removal of part of the lobe of the lung). Cancers in some locations (such as where the windpipe divides into the left and right main bronchi) may be treated with a sleeve resection, but in some cases they may be hard to remove completely without removing a lobe (lobectomy) or even an entire lung (pneumonectomy).For some stage 0 cancers, treatments s

stage3a + stage3b = stage 3

In [43]:
TestStage_v2[4] = TestStage_v2[4] + TestStage_v2[5]
del TestStage_v2[5]
TestStage_v2

['For these cancers, malignant cells are seen on sputum cytology but no obvious tumor can be found with bronchoscopy or imaging tests. They are usually early-stage cancers. Bronchoscopy and possibly other tests are usually repeated every few months to look for a tumor. If a tumor is found, treatment will depend on the stage.',
 'Because stage 0 NSCLC is limited to the lining layer of airways and has not invaded deeper into the lung tissue or other areas, it is usually curable by surgery alone. No chemotherapy or radiation therapy is needed.If you are healthy enough for surgery, you can usually be treated by segmentectomy or wedge resection (removal of part of the lobe of the lung). Cancers in some locations (such as where the windpipe divides into the left and right main bronchi) may be treated with a sleeve resection, but in some cases they may be hard to remove completely without removing a lobe (lobectomy) or even an entire lung (pneumonectomy).For some stage 0 cancers, treatments s

In [49]:
messages = [dataStage1_v2] + TestStage_v2
print(len(messages))
encoding_matrix = embed_fn(messages)
results = np.inner(encoding_matrix, encoding_matrix)
results[0]

8
['Treatments for stage 1 non–small cell lung cancer Surgery is a standard treatment for stage 1 non–small cell lung cancer for people who are well enough to have surgery.  Lobectomy to remove the lobe of the lung is the main type of surgery for stage 1 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed. Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 1 non–small cell lung cancer in people who do not have very good lung function. Sleeve resection is used to remove a tumour in the large airway of the lung (bronchus). During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. The surgery may be stopped if there is cancer in more lymph nodes than was shown with diagnostic tests. This is done because the cancer has spread too far for surgery to be helpful as a treatment.  Surgery may be done aga

array([0.9999999 , 0.81553197, 0.87515914, 0.8585496 , 0.89456177,
       0.8863358 , 0.79708266, 0.84831846], dtype=float32)

In [46]:
messages

['Treatments for stage 1 non–small cell lung cancer Surgery is a standard treatment for stage 1 non–small cell lung cancer for people who are well enough to have surgery.  Lobectomy to remove the lobe of the lung is the main type of surgery for stage 1 non–small cell lung cancer. It offers the best chance that the cancer will be completely removed. Wedge or segmental resection is used to remove the tumour along with a margin of healthy lung tissue. This type of surgery may be offered for stage 1 non–small cell lung cancer in people who do not have very good lung function. Sleeve resection is used to remove a tumour in the large airway of the lung (bronchus). During surgery for non–small cell lung cancer, the lymph nodes in the chest and around the lungs are removed. The surgery may be stopped if there is cancer in more lymph nodes than was shown with diagnostic tests. This is done because the cancer has spread too far for surgery to be helpful as a treatment.  Surgery may be done again

In [47]:
print(results[0][1:].argmax())
messages[results[0][1:].argmax()+1]

3


'People who have stage II NSCLC and are healthy enough for surgery usually have the cancer removed by lobectomy or sleeve resection. Sometimes removing the whole lung (pneumonectomy) is needed.Any lymph nodes likely to have cancer in them are also removed. The extent of lymph node involvement and whether or not cancer cells are found at the edges of the removed tissues are important factors when planning the next step of treatment.After surgery, the removed tissue is checked to see if there are cancer cells at the edges of the surgery specimen.  This might mean that some cancer has been left behind, so a second surgery might be done to try to remove any remaining cancer. This may be followed by chemotherapy (chemo). Another option is to treat with radiation, sometimes  with chemo.Even if positive margins are not found, chemo is usually recommended after surgery to try to destroy any cancer cells that might have been left behind. As with stage I cancers, newer lab tests now being studie

**Wrong stage 1 -> stage 2**

In [51]:
messages = [dataStage2_v2] + TestStage_v2
print(len(messages))
encoding_matrix = embed_fn(messages)
results = np.inner(encoding_matrix, encoding_matrix)
results[0]

8


array([1.        , 0.8161068 , 0.87506163, 0.86850774, 0.894032  ,
       0.88969296, 0.7958753 , 0.83801234], dtype=float32)

In [52]:
print(results[0][1:].argmax())
messages[results[0][1:].argmax()+1]

3


'People who have stage II NSCLC and are healthy enough for surgery usually have the cancer removed by lobectomy or sleeve resection. Sometimes removing the whole lung (pneumonectomy) is needed.Any lymph nodes likely to have cancer in them are also removed. The extent of lymph node involvement and whether or not cancer cells are found at the edges of the removed tissues are important factors when planning the next step of treatment.After surgery, the removed tissue is checked to see if there are cancer cells at the edges of the surgery specimen.  This might mean that some cancer has been left behind, so a second surgery might be done to try to remove any remaining cancer. This may be followed by chemotherapy (chemo). Another option is to treat with radiation, sometimes  with chemo.Even if positive margins are not found, chemo is usually recommended after surgery to try to destroy any cancer cells that might have been left behind. As with stage I cancers, newer lab tests now being studie

**Correct stage 2 -> stage 2**

In [53]:
messages = [dataStage3_v2] + TestStage_v2
print(len(messages))
encoding_matrix = embed_fn(messages)
results = np.inner(encoding_matrix, encoding_matrix)
results[0]

8
ERROR! Session/line number was not unique in database. History logging moved to new session 63


array([0.9999999 , 0.8585387 , 0.8633062 , 0.80717397, 0.8717682 ,
       0.9286721 , 0.858235  , 0.8740091 ], dtype=float32)

In [54]:
print(results[0][1:].argmax())
messages[results[0][1:].argmax()+1]

4


'Treatment for stage IIIA NSCLC may include some combination of radiation therapy, chemotherapy (chemo), and/or surgery. For this reason, planning treatment for stage IIIA NSCLC often requires input from a medical oncologist, radiation oncologist, and a thoracic surgeon. Your treatment options depend on the size of the tumor, where it is in your lung, which lymph nodes it has spread to, your overall health, and how well you are tolerating treatment.For patients who can tolerate it, treatment usually starts with chemo, often combined with radiation therapy (also called chemoradiation). Surgery may be an option after this if the doctor thinks any remaining cancer can be removed and the patient is healthy enough. (In some cases, surgery may be an option as the first treatment.) This is often followed by chemo, and possibly radiation therapy if it hasn’t been given before.For people who are not healthy enough for surgery, radiation therapy, which may be combined with chemo, is often used. 

**Correct stage 3 -> stage 3**

**=> it can chose correctly stage 2 and 3 in test web**

In [55]:
############################################################################

ERROR! Session/line number was not unique in database. History logging moved to new session 65


## Process the text and using LSA, LDA and NNMF to extract the important information

### Processing LSA
- Tokenize the text articles
- Remove stop words
- Perform stemming on text artcle

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
!pip install --upgrade gensim


     |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
#import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel, LdaModel, nmf
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [0]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # # add tokens to list
        texts.append(stopped_tokens)
    return texts

### Prepare Corpus
Next step is to prepare corpus. Here, you need to create a document-term matrix and dictionary of terms.

In [0]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

### Create an LSA model using Gensim
After corpus creation, a model is generated by using LSA.



In [0]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

In [0]:
def create_gensim_lda_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LDA model using gensim
    Output : return LDA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LDA model
    ldamodel = LdaModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(ldamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return ldamodel

In [0]:
def create_gensim_nmf_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create nmf model using gensim
    Output : return nmf model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate nmf model
    nmfmodel = nmf.Nmf(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(nmfmodel.print_topics(num_topics=number_of_topics, num_words=words))
    return nmfmodel

### Determine the number of topics


Another extra step needs to be taken to optimize results by identifying an optimum amount of topics.

In [0]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [0]:
def plot_graph(doc_clean,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix,doc_clean,
                                                            stop, start, step)
    # Show graph
    x = range(start, stop, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [92]:
# LSA Model
number_of_topics=1
words=10
document_list = [messages[results[0][1:].argmax()+1]]
clean_text=preprocess_data(document_list)
lsamodel=create_gensim_lsa_model(clean_text,number_of_topics,words)
ldamodel = create_gensim_lda_model(clean_text,number_of_topics,words)
nmfmodel = create_gensim_nmf_model(clean_text,number_of_topics,words)

[(0, '0.438*"treatment" + 0.328*"may" + 0.328*"radiation" + 0.255*"surgery" + 0.219*"chemo" + 0.219*"therapy" + 0.219*"often" + 0.146*"cancer" + 0.146*"chemoradiation" + 0.146*"considered"')]
[(0, '0.044*"treatment" + 0.034*"radiation" + 0.034*"may" + 0.027*"surgery" + 0.024*"often" + 0.024*"therapy" + 0.024*"chemo" + 0.017*"chemoradiation" + 0.017*"cancer" + 0.017*"considered"')]
[(0, '0.061*"treatment" + 0.046*"may" + 0.046*"radiation" + 0.036*"surgery" + 0.030*"therapy" + 0.030*"chemo" + 0.030*"often" + 0.020*"cancer" + 0.020*"chemoradiation" + 0.020*"considered"')]


In [93]:
words_topic = [x[0] for x in lsamodel.show_topic(0,topn=words) ]
' '.join(words_topic)

'treatment may radiation surgery chemo therapy often cancer chemoradiation considered'

In [94]:
words_topic = [x[0] for x in ldamodel.show_topic(0,topn=words) ]
' '.join(words_topic)

'treatment radiation may surgery often therapy chemo chemoradiation cancer considered'

In [95]:
words_topic = [x[0] for x in nmfmodel.show_topic(0,topn=words) ]
' '.join(words_topic)

'treatment may radiation surgery therapy chemo often cancer chemoradiation considered'

**=> LSA, LDA, NMF give nearly similar words in the topic of stage 2 on the test web**


## Using PageRank and Sequence2Sequence to summarize the text information

Text Summarisation with Gensim (TextRank algorithm)-

In [0]:
from gensim.summarization.summarizer import summarize
textRank = summarize(document_list[0], ratio = 0.5)
print(textRank)

Sometimes removing the whole lung (pneumonectomy) is needed.Any lymph nodes likely to have cancer in them are also removed.
This might mean that some cancer has been left behind, so a second surgery might be done to try to remove any remaining cancer.
Another option is to treat with radiation, sometimes  with chemo.Even if positive margins are not found, chemo is usually recommended after surgery to try to destroy any cancer cells that might have been left behind.


**=>- use TextRank to summarize (ration 50%) still keep important information**

## Conclusion

**- Combine all paragraphs of 1 stage will give better results than compare each paragraph of data with test data**

**- Use Google Universal sentence encoder to encode pargraph can determine area of information of stage 2 and stage 3**

**- LSA, LDA, NFM, give almost similar words in the topic of stage 2**

**- Use TextRank to summarize (ration 50%) still keep important information**

- Use Scipy in Jupyter Notebook: https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html
- Find next siblings until a certain one using beautifulsoup: https://stackoverflow.com/questions/11647348/find-next-siblings-until-a-certain-one-using-beautifulsoup

- Model for find similarity word, sentencem paragraph (*Google Universal sentence encoder*) https://towardsdatascience.com/use-cases-of-googles-universal-sentence-encoder-in-production-dd5aaab4fc15 

   https://github.com/sambit9238/Deep-Learning/blob/master/tensorflow_hub_useT.ipynb

  https://nlp.town/blog/sentence-similarity/

- Great NLP Project GitHub
https://github.com/susanli2016/NLP-with-Python/blob/master/Fuzzy%20String%20Matching.ipynb


- TREATMENT TRAINING DATA: https://www.cancer.ca/en/cancer-information/cancer-type/lung/treatment/stage-3/?region=on

  https://www.texasoncology.com/types-of-cancer/lung-cancer/non-small-cell-lung-cancer/stage-iiib-non-small-cell-lung-cancer

  https://www.healthline.com/health/lung-cancer#treatment

- EN | Non-Small-Cell Lung Cancer: Guide for Patients


- TREATMENT TEST DATA: https://www.cancer.org/cancer/lung-cancer/treating-non-small-cell/by-stage.html

- STAGE: https://www.cancer.org/cancer/lung-cancer/detection-diagnosis-staging/staging-nsclc.html

  https://www.cancer.org/treatment/treatments-and-side-effects/treatment-types.html


- LSA Datacamp: https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python



- Understand TextRank for Keyword Extraction by Python: https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0 

- Text Summarisation with Gensim (TextRank Algorithm):  https://medium.com/@shivangisareen/text-summarisation-with-gensim-textrank-46bbb3401289